In [30]:
from fenics import *

# Create mesh and define function space
n = 32
mesh = UnitSquareMesh(n, n)
W = FunctionSpace(mesh, 'P', 1) # pressure space
V = VectorFunctionSpace(mesh, 'P', 2)

# Set parameters
dt = 0.05
t = 0.0
epsilon = Constant(0.1)
append = False
kmin = 0.01
K = Expression('max(exp(-pow((x[1]-0.5-0.1*sin(10*x[0]))/0.1, 2)), 0.01)', degree=1)
phi = Constant(0.2)

# Define boundary condition
pD = Expression('10*(1-x[0])*t', t=0.0, degree=1)

def boundary(x, on_boundary):
    return on_boundary

def left(x, on_boundary):
    return on_boundary and near(x[0], 0.0)

bcs = [DirichletBC(W, pD, boundary)]
cbcs = [DirichletBC(W, Constant(1), left)]

p = TrialFunction(W)
q = TestFunction(W)
f = Constant(0.0)
a = K * inner(grad(p), grad(q))*dx
L = f*q*dx

u = Function(V)

c_n = Function(W)
c = Function(W)
s = TestFunction(W)
Form = (1/dt)*(c-c_n)*s*dx + dot(u, grad(c))*s*dx + epsilon*dot(grad(c), grad(s))*dx - f*s*dx


p = Function(W)

p_file = XDMFFile("perfusion/p.xdmf")
u_file = XDMFFile("perfusion/u.xdmf")
c_file = XDMFFile("perfusion/c.xdmf")

while t <= 1.0:
    pD.t = t
    solve(a == L, p, bcs=bcs)
    u.assign(project(-K*grad(p), V))
    solve(Form == 0, c, bcs=cbcs)
    c_n.assign(c)
    
    p_file.write_checkpoint(p, 'pressure', t, append=append)
    u_file.write_checkpoint(u, 'velocity', t, append=append)
    c_file.write_checkpoint(c, 'concentration', t, append=append)
    t += dt
    append = True
    
p_file.close()
u_file.close()
c_file.close()